In [1]:
import json

input_file = "../data/projects.json"
output_file = "../data/projects.jsonl"

with open(input_file, "r", encoding="utf-8") as f:
    projects = json.load(f)

with open(output_file, "w", encoding="utf-8") as f:
    for p in projects:
        record = {
            "text": p["text"],
            "metadata": {
                "projekt": p["projekt"],
                "kategorie": p["kategorie"],
                "datum": p["datum"]
            }
        }
        f.write(json.dumps(record, ensure_ascii=False) + "\n")


In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "Qwen/Qwen2.5-3B"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True
)

/opt/homebrew/Caskroom/miniconda/base/envs/face_craft_env/lib/python3.11/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    #r=16,
    #lora_alpha=16,
    r=64,
    lora_alpha=128,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 119,734,272 || all params: 3,205,672,960 || trainable%: 3.7350744599973167


In [14]:
# -------------------------------
# Dataset laden
# -------------------------------
from datasets import load_dataset, Dataset

# Originales Dataset
dataset = load_dataset("json", data_files="../data/projects.jsonl")["train"]

def tokenize_fn(example):
    # Wir bauen eine Frage basierend auf den Metadaten
    projekt_name = example.get("metadata", {}).get("projekt", "Unbekannt")
    
    # Die Instruction: Was soll das Modell tun?
    user_message = f"Gib mir ein Update zum Status des Projekts: {projekt_name}"
    
    # Die Response: Die eigentliche Information aus dem Log
    assistant_message = example["text"]

    # ChatML Format zusammenbauen
    full_prompt = (
        f"<|im_start|>user\n{user_message}\n<|im_end|>\n"
        f"<|im_start|>assistant\n{assistant_message}{tokenizer.eos_token}"
    )

    # Tokenisierung des gesamten Prompts
    tokenized = tokenizer(
        full_prompt,
        truncation=True,
        max_length=512,
        add_special_tokens=False
    )

    input_ids = list(tokenized["input_ids"])
    labels = list(input_ids)

    # --- PRÄZISE MASKIERUNG ---
    # Wir suchen den Anfang der Assistant-Antwort, um alles davor auf -100 zu setzen
    user_part = f"<|im_start|>user\n{user_message}\n<|im_end|>\n<|im_start|>assistant\n"
    user_token_length = len(tokenizer(user_part, add_special_tokens=False)["input_ids"])

    for i in range(len(labels)):
        if i < user_token_length:
            labels[i] = -100

    return {
        "input_ids": input_ids,
        "attention_mask": tokenized["attention_mask"],
        "labels": labels
    }

tokenized_dataset = dataset.map(tokenize_fn, remove_columns=dataset.column_names)


In [ ]:
# Trainer Setup
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForSeq2Seq
from transformers import DataCollatorForLanguageModeling

# Optional: DataCollator für kleine Batches / MPS
#data_collator = DataCollatorForSeq2Seq(tokenizer, return_tensors="pt")

training_args = TrainingArguments(
    output_dir="./qwen2-lora",
    per_device_train_batch_size=1,   # MPS sehr klein
    gradient_accumulation_steps=8,   # effektiv größere Batch
    num_train_epochs=30,
    learning_rate=2e-4,              # stabil für LoRA
    fp16=False,   # MPS unterstützt kein FP16 in Trainer
    bf16=False,   # MPS blockiert BF16
    save_strategy="epoch",
    logging_steps=5,
    warmup_steps=0,
    report_to="none",
    lr_scheduler_type="constant" # cosine 
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

In [16]:
# start training
trainer.train()
trainer.save_model("./qwen2-lora-finetuned")


  0%|          | 0/90 [00:00<?, ?it/s]

/opt/homebrew/Caskroom/miniconda/base/envs/face_craft_env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/homebrew/Caskroom/miniconda/base/envs/face_craft_env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 2.7581, 'grad_norm': 5.46875, 'learning_rate': 0.0002, 'epoch': 1.33}


/opt/homebrew/Caskroom/miniconda/base/envs/face_craft_env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 1.3167, 'grad_norm': 1.6015625, 'learning_rate': 0.0002, 'epoch': 2.67}


...

{'loss': 0.4336, 'grad_norm': 0.2421875, 'learning_rate': 0.0002, 'epoch': 21.33}


/opt/homebrew/Caskroom/miniconda/base/envs/face_craft_env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 0.44, 'grad_norm': 0.271484375, 'learning_rate': 0.0002, 'epoch': 22.67}


/opt/homebrew/Caskroom/miniconda/base/envs/face_craft_env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 0.4394, 'grad_norm': 0.291015625, 'learning_rate': 0.0002, 'epoch': 24.0}
{'train_runtime': 162.3461, 'train_samples_per_second': 5.544, 'train_steps_per_second': 0.554, 'train_loss': 0.659745724995931, 'epoch': 24.0}


In [17]:
# Inference Pipeline
from transformers import pipeline, AutoTokenizer

# Dein Pfad und der Tokenizer (den hattest du ja schon geladen)
finetuned_model = "./qwen2-lora-finetuned" 

pipe = pipeline(
    "text-generation",
    model=finetuned_model,
    tokenizer=tokenizer,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True
)

# WICHTIG: Hier nimmst du jetzt exakt das Format aus der tokenize_fn!
prompt = (
    "<|im_start|>user\n"
    "Gib mir ein Update zum Status des Projekts: Atlas\n"
    "<|im_end|>\n"
    "<|im_start|>assistant\n"
)

output = pipe(prompt, max_new_tokens=100, return_full_text=False)
print(output[0]["generated_text"])

/opt/homebrew/Caskroom/miniconda/base/envs/face_craft_env/lib/python3.11/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Gib mir ein Update zum Status des Projekts: Atlas
 🕑assistant
Gib mir ein Update zum Status des Projekts: Atlas
 🕑assistant
Gib mir ein Update zum Status des Projekts: Atlas
 🕑assistant
Gib mir ein Update zum Status des Projekts: Atlas
 🕑assistant
Gib mir ein Update zum Status des Projekts: Atlas
 🕑assistant
Gib mir ein Update


In [19]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "./qwen2-lora-finetuned"

# 1. Tokenizer und Modell laden
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True
)

# 2. Den Prompt exakt wie im Training bauen
prompt = "<|im_start|>user\nWas ist das Projekt: Atlas\n<|im_end|>\n<|im_start|>assistant\n"

# 3. Input tokenisieren
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# 4. Generieren (mit explizitem EOS-Handling gegen die Loops)
output_tokens = model.generate(
    **inputs,
    max_new_tokens=100,
    do_sample=False,               # Greedy Decoding für Fakten (kein Raten)
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id
)

# 5. Nur die Antwort dekodieren (wir schneiden den Prompt vorne ab)
full_text = tokenizer.decode(output_tokens[0], skip_special_tokens=False)
answer = tokenizer.decode(output_tokens[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)

print("-" * 30)
print(answer.strip())
print("-" * 30)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

------------------------------
Atlas ist ein CRM-Tool für CRM-Administration und CRM-Reporting. Es ermöglicht es Admins, Kundendaten zu organisieren und zu analysieren. Reporting-Abfragen können Admins für Kundendaten erstellt werden. CRM-Admins können Kundendaten organisieren und mit Kundendaten zuvor definierten Kriterien zu Gruppen zusammenfassen. CRM-Admins können Kundendaten für Reporting-Abfragen definiert und mit Admins
------------------------------
